# Import Necessary Libs
# (This is the demo code for the toy example: rating.csv. The main purpose of this demo code is to help students understand the KNN-based CF algorithms.)

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Load Dataset

In [2]:
names = ['user_id', 'item_id', 'rating']
df = pd.read_csv('rating.csv', sep='\t', names=names)
df.head()

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')
df

5 users
4 items


,user_id,item_id,rating
0,0,0,5
1,0,1,3
2,0,2,5
3,0,3,1
4,1,0,5
5,1,1,4
6,1,2,3
7,1,3,1
8,2,0,4
9,2,1,4


# Split Dataset into Training Dataset and Testing Dataset

In [3]:
train_df, test_df = train_test_split(df, test_size=0.1)
train_df, test_df

(    user_id  item_id  rating
 1         0        1       3
 18        4        2       3
 16        4        0       1
 17        4        1       2
 19        4        3       5
 4         1        0       5
 15        3        1       3
 6         1        2       3
 9         2        1       4
 5         1        1       4
 12        3        0       4
 7         1        3       1
 11        2        3       1
 14        3        3       5
 2         0        2       5
 13        3        2       5
 3         0        3       1
 8         2        0       4,
     user_id  item_id  rating
 0         0        0       5
 10        2        2       5)

In [4]:
# Training Dataset
train_ds = np.zeros((n_users, n_items))
for row in train_df.itertuples():
    train_ds[row[1], row[2]] = row[3]
train_ds = pd.DataFrame(train_ds)

# Testing Dataset
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    test_ds[row[1], row[2]] = row[3]
test_ds = pd.DataFrame(test_ds)

train_ds, test_ds

(     0    1    2    3
 0  0.0  3.0  5.0  1.0
 1  5.0  4.0  3.0  1.0
 2  4.0  4.0  0.0  1.0
 3  4.0  3.0  5.0  5.0
 4  1.0  2.0  3.0  5.0,
      0    1    2    3
 0  5.0  0.0  0.0  0.0
 1  0.0  0.0  0.0  0.0
 2  0.0  0.0  5.0  0.0
 3  0.0  0.0  0.0  0.0
 4  0.0  0.0  0.0  0.0)

# Fitting the Algorithm

## User-based

### Compute Pearson Correlation Coefficient for Each Pair of Users in Training Dataset

In [5]:
GAMMA = 0
EPSILON = 1e-9

np_user_pearson_corr = np.zeros((n_users, n_users))

for i, user_i_vec in enumerate(train_ds.values):
    for j, user_j_vec in enumerate(train_ds.values):
        print(str(i) + " user: " + str(user_i_vec) + "; " + str(j) + " user: " + str(user_j_vec))
        
        # ratings corated by the current pair of users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        print("co-reated item index" + str(corrated_index))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)
        
        print("np.clip(user_i_vec, 0, 1)" + " is " + str(np.clip(user_i_vec, 0, 1)))
        

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j
        
        print("mean_user_i " + " is " + str(mean_user_i))
        print("user_i_sub_mean " + " is " + str(user_i_sub_mean))

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = sim
        #weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim
        print("len(corrated_index) " + " is " + str(len(corrated_index)))

        np_user_pearson_corr[i][j] = weighted_sim
        
        ###
        #print(str(i) + " user: " + str(user_i_vec) + "; " + str(j) + " user: " + str(user_j_vec))
        #print("co-reated item index" + str(corrated_index))
        #print("np.clip(user_i_vec, 0, 1)" + " is " + str(np.clip(user_i_vec, 0, 1)))
        #print("np.clip(user_j_vec, 0, 1)" + " is " + str(np.clip(user_j_vec, 0, 1)))
        #print('i' + " user average: " + str(mean_user_i))
        #print('j' + " user average: " + str(mean_user_j))
np_user_pearson_corr

0 user: [0. 3. 5. 1.]; 0 user: [0. 3. 5. 1.]
co-reated item index[1 2 3]
np.clip(user_i_vec, 0, 1) is [0. 1. 1. 1.]
mean_user_i  is 2.999999999
user_i_sub_mean  is [ 1.00000008e-09  2.00000000e+00 -2.00000000e+00]
len(corrated_index)  is 3
0 user: [0. 3. 5. 1.]; 1 user: [5. 4. 3. 1.]
co-reated item index[1 2 3]
np.clip(user_i_vec, 0, 1) is [0. 1. 1. 1.]
mean_user_i  is 2.999999999
user_i_sub_mean  is [ 1.00000008e-09  2.00000000e+00 -2.00000000e+00]
len(corrated_index)  is 3
0 user: [0. 3. 5. 1.]; 2 user: [4. 4. 0. 1.]
co-reated item index[1 3]
np.clip(user_i_vec, 0, 1) is [0. 1. 1. 1.]
mean_user_i  is 2.999999999
user_i_sub_mean  is [ 1.00000008e-09 -2.00000000e+00]
len(corrated_index)  is 2
0 user: [0. 3. 5. 1.]; 3 user: [4. 3. 5. 5.]
co-reated item index[1 2 3]
np.clip(user_i_vec, 0, 1) is [0. 1. 1. 1.]
mean_user_i  is 2.999999999
user_i_sub_mean  is [ 1.00000008e-09  2.00000000e+00 -2.00000000e+00]
len(corrated_index)  is 3
0 user: [0. 3. 5. 1.]; 4 user: [1. 2. 3. 5.]
co-reated ite

array([[ 1.00000000e+00,  5.92999453e-01,  8.94427191e-01,
         5.39163910e-11, -5.92999453e-01],
       [ 5.92999453e-01,  1.00000000e+00,  9.69560705e-01,
        -6.62541349e-01, -1.00000000e+00],
       [ 8.94427191e-01,  9.69560705e-01,  1.00000000e+00,
        -8.28078671e-01, -9.69560705e-01],
       [ 5.39163910e-11, -6.62541349e-01, -8.28078671e-01,
         1.00000000e+00,  6.62541349e-01],
       [-5.92999453e-01, -1.00000000e+00, -9.69560705e-01,
         6.62541349e-01,  1.00000000e+00]])

### Predict Ratings

In [6]:
np_predictions = np.zeros((n_users, n_items))

K = 3
EPSILON = 1e-9

for (i, j), rating in np.ndenumerate(test_ds.values):
    if rating > 0:
        print(str(i) + " " + str(j))
        # find top-k most similar users as the current user, remove itself
        sim_user_ids = np.argsort(np_user_pearson_corr[i])[-(K + 1):-1]
        ### useful link for how to access values in numpy arrays: https://numpy.org/doc/stable/reference/arrays.indexing.html

        print(np.argsort(np_user_pearson_corr[i]))
        print(np_user_pearson_corr[i][:])
        print(np.argsort(np_user_pearson_corr[i])[-(K + 1):-1])
        # the coefficient values of similar users
        sim_val = np_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = train_ds.values[sim_user_ids]
        user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        #print(sim_users)
        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])

        # filter unrated items
        #w = np.clip(sim_users[mask_rated_j, j], 0, 1)
        #sim_r_sum_mean *= w
        #print(sim_users[:, j])
        
        np_predictions[i][j] = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
np_predictions

0 0
[4 3 1 2 0]
[ 1.00000000e+00  5.92999453e-01  8.94427191e-01  5.39163910e-11
 -5.92999453e-01]
[3 1 2]
2 2
[4 3 0 1 2]
[ 0.89442719  0.96956071  1.         -0.82807867 -0.96956071]
[3 0 1]


array([[4.29900607, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 3.89332654, 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

### Evaluation

#### Mean Absolute Error (MAE)

In [7]:
#==================MAE on Testing set===================#
labels = test_ds.values

# absolute error on all ratings
absolute_error = np.abs(np_predictions - labels)
print(absolute_error)

# weight
weight = np.clip(labels, 0, 1)
print(weight)

# absoulte error on rated ratings
abs_error = absolute_error * weight

# MAE
MAE = np.sum(abs_error) / np.sum(weight)

print("MAE on Tesing set (User-based): " + str(MAE));

[[0.70099393 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         1.10667346 0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
MAE on Tesing set (User-based): 0.9038336987360527


#### Root Mean Squared Error (RMSE)

In [8]:
#==================RMSE on Testing set===================
labels = test_ds.values

# squared error on all ratings
squared_error = np.square(np_predictions - labels)
weight = np.clip(labels, 0, 1)

# squared error on rated ratings
squared_error = squared_error * weight

# RMSE
RMSE = np.sqrt(np.sum(squared_error) / np.sum(weight))

print("RMSE on Tesing set (User-based): " + str(RMSE));

RMSE on Tesing set (User-based): 0.9263149166909437


In [9]:
print(train_ds.T.values)

[[0. 5. 4. 4. 1.]
 [3. 4. 4. 3. 2.]
 [5. 3. 0. 5. 3.]
 [1. 1. 1. 5. 5.]]


In [10]:
print(train_ds.values)

[[0. 3. 5. 1.]
 [5. 4. 3. 1.]
 [4. 4. 0. 1.]
 [4. 3. 5. 5.]
 [1. 2. 3. 5.]]


## Item-based

### Compute Pearson Correlation Coefficient for Each Pair of Users in Training Dataset

In [11]:
DELTA = 25
EPSILON = 1e-9

np_item_pearson_corr = np.zeros((n_items, n_items))

for i, item_i_vec in enumerate(train_ds.T.values):
    for j, item_j_vec in enumerate(train_ds.T.values):

        # ratings corated by the current pair od items
        mask_i = item_i_vec > 0
        mask_j = item_j_vec > 0

        # corrated index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of item_i_vec and item_j_vec
        mean_item_i = np.sum(item_i_vec) / (np.sum(np.clip(item_i_vec, 0, 1)) + EPSILON)
        mean_item_j = np.sum(item_j_vec) / (np.sum(np.clip(item_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        item_i_sub_mean = item_i_vec[corrated_index] - mean_item_i
        item_j_sub_mean = item_j_vec[corrated_index] - mean_item_j

        r_ui_sub_ri_sq = np.square(item_i_sub_mean)
        r_uj_sub_rj_sq = np.square(item_j_sub_mean)

        r_ui_sub_ri_sq_sum_sqrt = np.sqrt(np.sum(r_ui_sub_ri_sq))
        r_uj_sub_rj_sq_sum_sqrt = np.sqrt(np.sum(r_uj_sub_rj_sq))

        sim = np.sum(item_i_sub_mean * item_j_sub_mean) / (r_ui_sub_ri_sq_sum_sqrt * r_uj_sub_rj_sq_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), DELTA) / DELTA) * sim

        np_item_pearson_corr[i][j] = weighted_sim

np_item_pearson_corr

array([[ 1.60000000e-01,  1.44463024e-01,  3.51324026e-02,
        -1.04595272e-01],
       [ 1.44463024e-01,  2.00000000e-01, -4.35464879e-11,
        -1.52752523e-01],
       [ 3.51324026e-02, -4.35464879e-11,  1.60000000e-01,
         3.13785842e-11],
       [-1.04595272e-01, -1.52752523e-01,  3.13785842e-11,
         2.00000000e-01]])

### Predict Ratings

In [12]:
np_predictions = np.zeros((n_users, n_items))

K = 10
EPSILON = 1e-9

for (i, j), rating in np.ndenumerate(test_ds.values):
    if rating > 0:
        # find top-k most similar items as the current item, remove itself
        sim_item_ids = np.argsort(np_item_pearson_corr[j])[-(K + 1):-1]

        # the coefficient values of similar items
        sim_val = np_item_pearson_corr[i][sim_item_ids]

        # the average value of the current item's ratings
        sim_items = train_ds.T.values[sim_item_ids]
        item_mean = np.sum(train_ds.T.values[j]) / (np.sum(np.clip(train_ds.T.values[j], 0, 1)) + EPSILON)
        sim_item_mean = np.sum(sim_items, axis=1) / (np.sum(np.clip(sim_items, 0, 1), axis=1) + EPSILON)

        # sim(u, v) * (r_v - mean_v)
        sim_r_sum_mean = sim_val * (sim_items[:, i] - sim_item_mean) 

        # filter unrated items
        w = np.clip(sim_items[:, i], 0, 1)
        sim_r_sum_mean *= w

        if np.sum(sim_val) == 0:
            np_predictions[i][j] = item_mean
        else:
            np_predictions[i][j] = item_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val) + EPSILON)
            
        np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
    

### Evaluation

#### Mean Absolute Error (MAE)

In [13]:
#==================MAE on Testing set===================#
labels = test_ds.values

# absolute error on all ratings
absolute_error = np.abs(np_predictions - labels)

# weight
weight = np.clip(labels, 0, 1)

# absoulte error on rated ratings
abs_error = absolute_error * weight

# MAE
MAE = np.sum(abs_error) / np.sum(weight)

print("MAE on Tesing set (Item-based): " + str(MAE));

MAE on Tesing set (Item-based): 0.2500000083021723


#### Root Mean Squared Error (RMSE)

In [14]:
#==================RMSE on Testing set===================#
labels = test_ds.values

# squared error on all ratings
squared_error = np.square(np_predictions - labels)
weight = np.clip(labels, 0, 1)

# squared error on rated ratings
squared_error = squared_error * weight

# RMSE
RMSE = np.sqrt(np.sum(squared_error) / np.sum(weight))

print("RMSE on Tesing set (Item-based): " + str(RMSE));

RMSE on Tesing set (Item-based): 0.3535534023343184
